In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

## Data Define - using DataLoader

In [3]:
batch_size = 64
test_batch_size = 64

train_loader = torch.utils.data.DataLoader(
                datasets.MNIST('dataset/', train=True, download=True, 
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=(0.5,), std=(0.5,))
                              ])),
                batch_size=batch_size,
                shuffle=True)

test_loader = torch.utils.data.DataLoader(
                datasets.MNIST('dataset/', train=False, 
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=(0.5,), std=(0.5,))
                              ])),
                batch_size=test_batch_size,
                shuffle=True)

/home/ziippy/venv_tf2/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        # feature extraction
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        
        #print(x.shape)
        # fully-connected
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
model = Net().to(device)

In [6]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)

In [7]:
epochs = 2
log_interval = 100

for epoch in range(1, epochs+1):
    # Train mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} {}/{} ({:.0f}%)\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), 
                loss.item()))
    
    # Evaluation mode
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()    
            pred = output.argmax(dim=1, keepdim=True)    
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {} / {} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))

/home/ziippy/venv_tf2/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 1 0/60000 (0%)	Loss: 2.311277
Train Epoch: 1 6400/60000 (11%)	Loss: 2.293156
Train Epoch: 1 12800/60000 (21%)	Loss: 2.279752
Train Epoch: 1 19200/60000 (32%)	Loss: 2.276142
Train Epoch: 1 25600/60000 (43%)	Loss: 2.239128
Train Epoch: 1 32000/60000 (53%)	Loss: 2.232791
Train Epoch: 1 38400/60000 (64%)	Loss: 2.202166
Train Epoch: 1 44800/60000 (75%)	Loss: 2.144686
Train Epoch: 1 51200/60000 (85%)	Loss: 2.131424
Train Epoch: 1 57600/60000 (96%)	Loss: 2.066348

Test set: Average loss: 2.0185, Accuracy: 6961 / 10000 (70%)
Train Epoch: 2 0/60000 (0%)	Loss: 2.009079
Train Epoch: 2 6400/60000 (11%)	Loss: 1.872707
Train Epoch: 2 12800/60000 (21%)	Loss: 1.658992
Train Epoch: 2 19200/60000 (32%)	Loss: 1.597974
Train Epoch: 2 25600/60000 (43%)	Loss: 1.400004
Train Epoch: 2 32000/60000 (53%)	Loss: 1.186932
Train Epoch: 2 38400/60000 (64%)	Loss: 0.966133
Train Epoch: 2 44800/60000 (75%)	Loss: 0.777622
Train Epoch: 2 51200/60000 (85%)	Loss: 0.819548
Train Epoch: 2 57600/60000 (96%)	Loss: